### Preparing Working Env

In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow_datasets as tfds
from importlib.util import find_spec
if find_spec("core") is None:
    import sys
    sys.path.append('..')
import tensorflow as tf
import numpy as np
import random
import os
from core.models.simclr_model import Pretrained_SimCLR_Model
from core.models.model_utils.lr_schedule import WarmUpAndCosineDecay
from core.datasets.data_augmentation import preprocess_image, get_config, train_classification_aug
import tensorflow_addons as tfa

In [2]:
# Set the random seeds
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

### Utility Functions

In [3]:
# def train_data_aug(img, lb):
#     xs = []
#     for _ in range(2):
#         xs.append(preprocess_image(img, config['IMG_SIZE'], config['IMG_SIZE'],
#                                    is_training=True,
#                                    color_distort=True,
#                                    test_crop=False))
#     img = tf.concat(xs, -1)
#     return img, tf.one_hot(lb, config['num_classes'])


# def val_data_aug(img, lb):
#     img = preprocess_image(img, config['IMG_SIZE'], config['IMG_SIZE'],
#                            is_training=False,
#                            color_distort=True,
#                            test_crop=False)
#     return img, lb

def train_data_aug(img, lb):
    xs = []
    for _ in range(2):
        xs.append(train_classification_aug(img, lb, img_size = config['IMG_SIZE'])[0])
    img = tf.concat(xs, -1)
    return img, tf.one_hot(lb, 4)

def val_data_aug(img, lb):
    xs = []
    for _ in range(2):
        xs.append(train_classification_aug(img, lb, img_size = config['IMG_SIZE'])[0])
    img = tf.concat(xs, -1)
    return img, tf.one_hot(lb, 4)

def resize_only(img, lb):
    img = train_classification_aug(img, lb, img_size = config['IMG_SIZE'])[0]
    return img, tf.one_hot(lb, config['num_classes'])

### Preparing the Data

In [4]:
# Load Retinal Data
ds_train, ds_train_info = tfds.load('RetinaDataset', split='train', shuffle_files=True, as_supervised=True, with_info=True)
ds_test  = tfds.load('RetinaDataset', split='test', shuffle_files=True, as_supervised=True)

### Building the SimCLR Model

In [5]:
config={
     "learning_rate": 0.01,
     "epochs": 10,
     "learning_rate_scaling": 'linear',
     "num_examples": len(ds_train),
     "warmup_epochs": 10,
     "batch_size": 256,
     "num_classes": 4,
     "use_blur": True,
     # ProjectionLayer Parameters
     "proj_head_mode": 'nonlinear',
     "proj_out_dim" : 128,
     "num_proj_layers": 3, 
     "ft_proj_selector": 0,
     # Resnet_parameter
     "resnet_depth": 18,
     "resnet_width_multiplier": 1,
     "resnet_se_ratio": 0.0,
     "resnet_sk_ratio": 0.0,
     # contrastive loss parameter
     "hidden_norm": True,
     "temperature" :1.0,
     # Image Size
     'IMG_SIZE': 128,
    }

In [6]:
# Initialize the model
model = Pretrained_SimCLR_Model(num_classes=config['num_classes'],
                                use_blur = config['use_blur'],
                                proj_head_mode = config['proj_head_mode'],
                                proj_out_dim = config['proj_out_dim'],
                                num_proj_layers = config['num_proj_layers'],
                                ft_proj_selector = config['ft_proj_selector'],
                                resnet_depth = config['resnet_depth'],
                                resnet_width_multiplier = config['resnet_width_multiplier'],
                                resnet_se_ratio = config['resnet_se_ratio'],
                                resnet_sk_ratio = config['resnet_sk_ratio'],
                                hidden_norm = config['hidden_norm'],
                                temperature = config['temperature'])
# Build the Model
input_shape_base   = (None, config['IMG_SIZE'], config['IMG_SIZE'], 3)
input_shape_simclr = (None, config['IMG_SIZE'], config['IMG_SIZE'], 6)
model.base_model.build(input_shape_base)
model.build(input_shape_simclr)
model.summary()

Model: "pretrained__sim_clr__model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet (Resnet)              multiple                  11190464  
_________________________________________________________________
head_supervised (SupervisedH multiple                  2052      
_________________________________________________________________
projection_head (ProjectionH multiple                  594304    
_________________________________________________________________
contrast_loss (Mean)         multiple                  2         
_________________________________________________________________
contrast_acc (Mean)          multiple                  2         
_________________________________________________________________
contrast_entropy (Mean)      multiple                  2         
_________________________________________________________________
supervised_loss (Mean)       multiple   

### Training the Model

In [7]:
# Define Scheduler and Optimizer
lr_scheduler = WarmUpAndCosineDecay(config['learning_rate'], 
                                    num_examples= config['num_examples'], 
                                    train_epochs= config['epochs'], 
                                    train_batch_size = config['batch_size'],
                                    learning_rate_scaling = config['learning_rate_scaling'], 
                                    warmup_epochs = config['warmup_epochs'])
optimizer = tfa.optimizers.LAMB(lr_scheduler,
                                weight_decay_rate=1e-6,
                                exclude_from_weight_decay=['batch_normalization', 'bias', 'head_supervised'])

model.compile(optimizer= optimizer)

model.fit(ds_train.map(train_data_aug).batch(config['batch_size'] ),
          epochs= config['epochs'])

# model.fit(ds_train.map(train_data_aug).batch(config['batch_size'] ),
#           epochs= config['epochs'],
#           validation_data = ds_test.map(val_data_aug).batch(config['batch_size']))

C:\Users\mohit\Anaconda3\envs\retina_env\lib\site-packages\tensorflow\python\autograph\impl\api.py:493: UserWarning: fill_value is not supported and is always 0 for TensorFlow < 2.4.0.
  return py_builtins.overload_of(f)(*args)


Epoch 1/10
  2/327 [..............................] - ETA: 1:50 - contrast_loss: 12.5684 - contrast_acc: 0.0059 - contrast_entropy: 5.4907 - supervised_loss: 1.3626 - supervised_acc: 0.3457WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.2090s vs `on_train_batch_end` time: 0.4570s). Check your callbacks.


327/327 [==============================] - 265s 811ms/step - contrast_loss: 12.3772 - contrast_acc: 0.0095 - contrast_entropy: 5.4909 - supervised_loss: 1.2427 - supervised_acc: 0.4402
Epoch 2/10
327/327 [==============================] - 262s 802ms/step - contrast_loss: 11.9519 - contrast_acc: 0.0215 - contrast_entropy: 5.4111 - supervised_loss: 1.2117 - supervised_acc: 0.4698
Epoch 3/10
327/327 [==============================] - 262s 801ms/step - contrast_loss: 11.6867 - contrast_acc: 0.0385 - contrast_entropy: 5.4065 - supervised_loss: 1.0908 - supervised_acc: 0.5630
Epoch 4/10
327/327 [==============================] - 262s 801ms/step - contrast_loss: 11.4951 - contrast_acc: 0.0614 - contrast_entropy: 5.4103 - supervised_loss: 0.9346 - supervised_acc: 0.6368
Epoch 5/10
327/327 [==============================] - 262s 802ms/step - contrast_loss: 11.2895 - contrast_acc: 0.1257 - contrast_entropy: 5.4261 - supervised_loss: 0.8365 - supervised_acc: 0.6826
Epoch 6/10
327/327 [===========

In [9]:
score = model.evaluate(ds_test.map(train_data_aug).batch(128))

8/8 [==============================] - 2s 300ms/step - contrast_loss: 9.5497 - contrast_acc: 0.4289 - contrast_entropy: 4.7033 - supervised_loss: 0.8210 - supervised_acc: 0.6672


In [10]:
score

[9.549652099609375,
 0.4289279580116272,
 4.703300476074219,
 0.8209872245788574,
 0.6672092080116272,
 9.549652099609375,
 0.4289279580116272,
 4.703300476074219,
 0.8209872245788574,
 0.6672092080116272]

### END